Λαμπρόπουλος Κωνσταντίνος $$ $$
ΑΜ : 1115201800092

Imports

In [197]:
import pandas as pd
import numpy as np
from PIL import Image
import glob
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

Load Image

In [198]:
def LoadImages(path,set_number):
    loadedimages = []
    labels = []
    for i in range(1,65):
        labels.append(f"{set_number}_{i}")
        if set_number < 10:
            if i < 10:
                image_path = f"{path}/person0{set_number}_0{i}.png"
            else:
                image_path = f"{path}/person0{set_number}_{i}.png"
            image = Image.open(image_path)
            loadedimages.append(np.array(image).flatten())
        else:
            if i < 10:
                image_path = f"{path}/person{set_number}_0{i}.png"
            else:
                image_path = f"{path}/person{set_number}_{i}.png"
            image = Image.open(image_path)
            loadedimages.append(np.array(image).flatten())   
    return (np.array(loadedimages),labels)


Perform PCA

In [199]:
def PerformPCA(X,num_components):
    pca = PCA(n_components=num_components,whiten=True)
    transformed_data = pca.fit_transform(X)
    eigen_vectors = pca.components_
    eigen_values = pca.explained_variance_
    return (transformed_data,eigen_vectors)

Eigen Space

In [200]:
def ProjectToEigenSpace(new_data,train_data,eigenvectors):
    # Calculate the mean face from the training data
    mean_face = np.mean(train_data, axis=0)

    # Subtract the mean face from the new data point
    centered_data = new_data - mean_face

    # Project the centered data point onto the eigenvectors
    projected_data = np.dot(centered_data, eigenvectors.T)

    return projected_data


Face Recognition

In [201]:
def FaceRecognition(train_data, train_labels,test_data):
    knn = NearestNeighbors(n_neighbors=1)
    knn.fit(train_data,train_labels)
    distances, indices = knn.kneighbors(test_data)
    predictions = np.array(train_labels)[indices]
    return predictions

main

In [207]:
def TrainEigenfaces(train_data, train_labels, d):
    # Perform PCA on the training data
    pca = PCA(n_components=d)
    transformed_train_data = pca.fit_transform(train_data)

    # Train a classifier (e.g., nearest neighbor)
    classifier = KNeighborsClassifier(n_neighbors=1)
    classifier.fit(transformed_train_data, train_labels)

    return pca, classifier

def TestEigenfaces(pca, classifier, test_data):
    # Project test data onto the eigenvectors
    transformed_test_data = pca.transform(test_data)

    # Predict labels for the projected test data
    predicted_labels = classifier.predict(transformed_test_data)

    return predicted_labels

def CalculateAccuracy(true_labels, predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels) * 100
    return accuracy

# Load training data and labels for Set_1
train_data, train_labels = LoadImages('./faces', set_number=1)

# Train Eigenfaces with d=9
pca_9, classifier_9 = TrainEigenfaces(train_data, train_labels, d=9)

# Train Eigenfaces with d=30
pca_30, classifier_30 = TrainEigenfaces(train_data, train_labels, d=30)

# Evaluate on Sets 1 to 5
for set_number in range(1, 6):
    # Load test data and labels for the current set
    test_data, test_labels = LoadImages('./faces', set_number=set_number)

    # Test Eigenfaces with d=9
    predicted_labels_9 = TestEigenfaces(pca_9, classifier_9, test_data)
    accuracy_9 = CalculateAccuracy(test_labels, predicted_labels_9)

    # Test Eigenfaces with d=30
    predicted_labels_30 = TestEigenfaces(pca_30, classifier_30, test_data)
    accuracy_30 = CalculateAccuracy(test_labels, predicted_labels_30)

    print(f"Set_{set_number}:")
    print(f"- Accuracy (d=9): {accuracy_9:.2f}%")
    print(f"- Accuracy (d=30): {accuracy_30:.2f}%")


Set_1:
- Accuracy (d=9): 100.00%
- Accuracy (d=30): 100.00%
Set_2:
- Accuracy (d=9): 0.00%
- Accuracy (d=30): 0.00%
Set_3:
- Accuracy (d=9): 0.00%
- Accuracy (d=30): 0.00%
Set_4:
- Accuracy (d=9): 0.00%
- Accuracy (d=30): 0.00%
Set_5:
- Accuracy (d=9): 0.00%
- Accuracy (d=30): 0.00%
